In [1]:
station_file = "model_results/2014-wqm-coldstart/ssm_station.out"
dest = "model_results/stations_cold.csv.gz"

water_vars = 52
bottom_vars = 104

from collections import deque
import numpy as np
import pandas as pd
from IPython.display import clear_output

In [2]:
with open(station_file) as fp:
    # The first line is just a label
    next(fp)
    nstation, nlayer = np.loadtxt([next(fp)]).astype(int)
    #print("Nstation", nstation)
    #print("Nlayer", nlayer)
    
    def get_list_of_variable_names_from_line(line):
        return line.replace("Variables=", "").replace("\"", "").rstrip().split(",")
    variables_list = get_list_of_variable_names_from_line(next(fp))
    #print(variables_list)
    variables_list.insert(0, "Time")

    data = {}
    for v in variables_list:
        data[v] = []
    times = []

    def read_block(varct, t):
        block = []
        for i, v in enumerate(variables_list):
            if v == 'Time':
                data[v].append(t)
                continue
            # The extra three is for the station, node, and layer
            if i >= varct + 3:
                # Fill in empty data that's not applicable to this layer
                data[v].append(np.nan)
                continue
            if len(block) == 0:
                block = deque(np.genfromtxt([next(fp)], missing_values='*************'))
            data[v].append(block.popleft())

    try:
        while True:
            # Read the number of stations/layers and the time
            istation, ilayers, t = np.loadtxt([next(fp)])
            istation = int(istation)
            ilayers = int(ilayers)
            times.append(t)
            clear_output(wait = True)
            print("TIME", t)
            for s in range(istation):
                for l in range(ilayers-1):
                    read_block(water_vars, t)
                read_block(bottom_vars, t)
    except StopIteration:
        pass

TIME 365.0


In [3]:
df = pd.DataFrame(data)
df.head()

,Time,StationID,Node,Layer,depth(m),DO,NO3,NH4,Alg1,Alg2,...,POC23,PON21,PON22,PON23,POP21,POP22,POP23,POS2,H1,BEN_STR
0,0.000463,1.0,6151.0,1.0,0.323122,7.99999,0.43,0.002,0.012996,0.012998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000463,1.0,6151.0,2.0,1.237050,7.99999,0.43,0.002,0.013000,0.013000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000463,1.0,6151.0,3.0,2.592920,7.99999,0.43,0.002,0.013000,0.013000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000463,1.0,6151.0,4.0,4.263970,7.99999,0.43,0.002,0.013000,0.013000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000463,1.0,6151.0,5.0,6.197590,7.99999,0.43,0.002,0.013000,0.013000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Extract the station node IDs to a separate dataframe
station_nodes = df[["StationID","Node"]].groupby('StationID')["Node"].first()
del df['StationID']
station_nodes.head()

StationID
1.0     6151.0
2.0     7786.0
3.0    11793.0
4.0     4040.0
5.0     5112.0
Name: Node, dtype: float64

In [5]:
# Fix dtype for node and layer
df['Node'] = df['Node'].astype(int)
df['Layer'] = df['Layer'].astype(int)
# Build the MultiIndex for time/node/layer
mi = pd.MultiIndex.from_frame(df[["Time","Node","Layer"]])
del df['Time']
del df['Node']
del df['Layer']
df.set_index(mi, inplace=True)
df.head()

depth(m)       DO   NO3    NH4      Alg1      Alg2  \
Time     Node Layer                                                       
0.000463 6151 1      0.323122  7.99999  0.43  0.002  0.012996  0.012998   
              2      1.237050  7.99999  0.43  0.002  0.013000  0.013000   
              3      2.592920  7.99999  0.43  0.002  0.013000  0.013000   
              4      4.263970  7.99999  0.43  0.002  0.013000  0.013000   
              5      6.197590  7.99999  0.43  0.002  0.013000  0.013000   

                         LDOC  RDOC      LPOC      RPOC  ...  POC23  PON21  \
Time     Node Layer                                      ...                 
0.000463 6151 1      0.499999   0.5  0.099641  0.099642  ...    NaN    NaN   
              2      0.499999   0.5  0.099999  0.100000  ...    NaN    NaN   
              3      0.499999   0.5  0.099999  0.100000  ...    NaN    NaN   
              4      0.499999   0.5  0.099999  0.100000  ...    NaN    NaN   
              5      0.499999   0.5  0.099999  0.100000  ...    NaN    NaN   

                     PON22  PON23  POP21  POP22  POP23  POS2  H1  BEN_STR  
Time     Node Layer                                                        
0.000463 6151 1        NaN    NaN    NaN    NaN    NaN   NaN NaN      NaN  
              2        NaN    NaN    NaN    NaN    NaN   NaN NaN      NaN  
              3        NaN    NaN    NaN    NaN    NaN   NaN NaN      NaN  
              4        NaN    NaN    NaN    NaN    NaN   NaN NaN      NaN  
              5        NaN    NaN    NaN    NaN    NaN   NaN NaN      NaN  

[5 rows x 104 columns]

Save all of the output to a CSV, compressed.
There is probably too much data for Excel to handle it as one gigantic file; see below if you want to get a spreadsheet of just one variable for one station.

In [6]:
df.to_csv(dest, compression="gzip")

Example of how to save one station's output to Excel

In [7]:
node_to_save = 40
var_to_save = "DO"

# All times, just this station
#      |         /  all layers
#      |        /       |   just this variable
#      |       /        |         /
df.loc[:, node_to_save, :][var_to_save]

Time        Layer
0.000463    1        8.00000
            2        8.00000
            3        8.00000
            4        8.00000
            5        8.00000
                      ...   
365.000000  6        5.05965
            7        4.99712
            8        4.70122
            9        4.73141
            10       4.71667
Name: DO, Length: 14610, dtype: float64

In [8]:
df.loc[:, node_to_save, :][var_to_save].to_excel("notebook_outs/node40_do.xlsx")